# CAPSTONE PROJECT - CLUSTERING MEXICO'S TOP 100 CITIES BY DEPRESSION RISK FACTORS
______________________________________________________________________________________________________________________________________________

### IMPORT LIBRARIES

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import types
from botocore.client import Config
import ibm_boto3
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## RETRIEVE AND CLEAN DATA
#### Data comes from 4 sources
 (INEGI) Mexico's National Institute of Statistics and Geography - Microdata-CSV Tables
 
 (INEGI) Mexico's National Institute of Statistics and Geography  - General Data - Mexican States - API Query
 
 (INEGI) Mexico's National Institute of Statistics and Geography  - General Data - Mexican Municipalities - API Query
 
 Foursquare - General Data - API Query/

### STEP 1 - INEGI - CSV TABLES DATA RETRIEVE AND CLEAN
#### The following tables were queried directly from INEGI Website and were stored in IBM Watson Study as assets

##### We will import first a table with the 116 most populated municipalities in Mexico. Then we will drop some columns and add the Features Column. In this case, every feature is a Risk Factor that increases the chances to trigger depression on any individual

In [66]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ddc00bb441b84f7792be0eedd381d2e5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='nT4_hoHzvyrqCFt5NA4Cd5LHkCNC4KJw3HfZ0GooM-zq',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Most Populated Municipalities.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_municipality = pd.read_csv(body)
df_municipality.head()


,State Index,State,Municipality Index,Municipality
0,1,Aguascalientes,1001,Aguascalientes
1,2,Baja California,2001,Ensenada
2,2,Baja California,2002,Mexicali
3,2,Baja California,2004,Tijuana
4,3,Baja California Sur,3003,La Paz


#### Now we will begin adding features from all of the other tables

In [67]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Total Population by State.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_population_state = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Total population with access to water.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_SF_No_Access_Water = pd.read_csv(body)


body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='New Mental Disorders Registered.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_G_New_Cases_Depression= pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Percentage of Population who has felt depression.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_G_Depression = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='People migrating in and out of the region.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_SE_Migrating_Out = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Percentage of people non affiliated to social security.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_SF_No_Social_Security = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Total 6 people or more sharing housing.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_SF_Housing_Overcrowding = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Population mortality by breast cancer.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_SI_Mortality_Breast_Cancer = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Population mortaility by prostate cancer.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_SI_Mortality_Prostate_Cancer = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='Population mortality by diabetes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_SI_Mortality_Diabetes = pd.read_csv(body)

print('All CSV Columns loaded successfully')

All CSV Columns loaded successfully


#### Merge all columns and then drop unnecesary columns to our purpose


In [4]:
#Merge Columns, they all have State Index and State in Common
df_risk_factors_1 = df_population_state 
dataframes= [df_G_New_Cases_Depression,df_SF_No_Access_Water,df_G_Depression,df_SE_Migrating_Out,df_SF_No_Social_Security,
             df_SI_Mortality_Breast_Cancer,df_SI_Mortality_Prostate_Cancer,df_SI_Mortality_Diabetes]
for dataframe in dataframes:
    df_risk_factors_1 =pd.merge(df_risk_factors_1,dataframe, on=['State Index', 'State'])
    
df_risk_factors_1.head()

,State Index,State,Total Population,New Depression Cases (No per 100 thousand inhabitants),New Anorexia & Bulimia Cases (No per 100 thousand inhabitants),Total_x,People with access inside home,Percentage of population who has felt depressed,Migrating out,Migrating in,Total_y,Afiliated,Non Afiliated,Non Specified,2016,Total Mortality by Prostate Cancer,Total Population mortality by diabetes
0,1,Aguascalientes,1316032,"107,32","1,58",1300635,1220535,"20,2",233073,136354,100,"86,9","12,9","0,2",71,69,801
1,2,Baja California,3348898,"144,81","5,11",3204661,3000950,"21,2",1299773,185457,100,"81,6","18,1","0,3",192,164,2427
2,3,Baja California Sur,718384,"112,67","2,2",658138,557229,"21,2",246685,33074,100,"85,9","13,6","0,5",34,36,377
3,4,Campeche,902250,"139,15","1,03",840907,534468,"31,3",180252,109734,100,"87,8","11,9","0,2",35,28,723
4,5,Coahuila de Zaragoza,2961708,"31,23","0,57",2901261,2633210,"45,6",362707,434617,100,"85,4","14,1","0,5",222,136,2744


In [5]:
#Drop Unnecessary columns
df_risk_factors_1= df_risk_factors_1.drop(['Total_y','Afiliated','Non Specified','Total_x'],axis=1)
df_risk_factors_1.head()

,State Index,State,Total Population,New Depression Cases (No per 100 thousand inhabitants),New Anorexia & Bulimia Cases (No per 100 thousand inhabitants),People with access inside home,Percentage of population who has felt depressed,Migrating out,Migrating in,Non Afiliated,2016,Total Mortality by Prostate Cancer,Total Population mortality by diabetes
0,1,Aguascalientes,1316032,"107,32","1,58",1220535,"20,2",233073,136354,"12,9",71,69,801
1,2,Baja California,3348898,"144,81","5,11",3000950,"21,2",1299773,185457,"18,1",192,164,2427
2,3,Baja California Sur,718384,"112,67","2,2",557229,"21,2",246685,33074,"13,6",34,36,377
3,4,Campeche,902250,"139,15","1,03",534468,"31,3",180252,109734,"11,9",35,28,723
4,5,Coahuila de Zaragoza,2961708,"31,23","0,57",2633210,"45,6",362707,434617,"14,1",222,136,2744


#### Rename columns to make their title clear

In [6]:
#Rename Columns to make their title clear
df_risk_factors_1 = df_risk_factors_1.rename(columns={"Total Population": "Total Population in the State", 
                                                      "People with access inside home": "People with Water access at home",
                                                      "Migrating out": "People migrating out of the State",
                                                      "Migrating in": "People migrating inside the State",
                                                      "Non Afiliated": "People without social security",
                                                      "2016": "Total Mortality by breast cancer"})
df_risk_factors_1.head(1)

,State Index,State,Total Population in the State,New Depression Cases (No per 100 thousand inhabitants),New Anorexia & Bulimia Cases (No per 100 thousand inhabitants),People with Water access at home,Percentage of population who has felt depressed,People migrating out of the State,People migrating inside the State,People without social security,Total Mortality by breast cancer,Total Mortality by Prostate Cancer,Total Population mortality by diabetes
0,1,Aguascalientes,1316032,"107,32","1,58",1220535,"20,2",233073,136354,"12,9",71,69,801


In [7]:
#From the columns, New Depression CAses, New Anorexia & Bulimia Cases, Percentage of population who has felt depressed, and people without social security
#are object type, which is not wrong, but it will be better to have them as int64 for further data manipulation
#To change types will have first to change commas by points for decimals

df_risk_factors_1["New Depression Cases (No per 100 thousand inhabitants)"]= [x.replace(',', '.') for x in df_risk_factors_1["New Depression Cases (No per 100 thousand inhabitants)"]]
df_risk_factors_1["New Depression Cases (No per 100 thousand inhabitants)"] = df_risk_factors_1["New Depression Cases (No per 100 thousand inhabitants)"].astype(float)

df_risk_factors_1["New Anorexia & Bulimia Cases (No per 100 thousand inhabitants)"]= [x.replace(',', '.') for x in df_risk_factors_1["New Anorexia & Bulimia Cases (No per 100 thousand inhabitants)"]]
df_risk_factors_1["New Anorexia & Bulimia Cases (No per 100 thousand inhabitants)"] = df_risk_factors_1["New Anorexia & Bulimia Cases (No per 100 thousand inhabitants)"].astype(float)

df_risk_factors_1["Percentage of population who has felt depressed"]= [x.replace(',', '.') for x in df_risk_factors_1["Percentage of population who has felt depressed"]]
df_risk_factors_1["Percentage of population who has felt depressed"] = df_risk_factors_1["Percentage of population who has felt depressed"].astype(float)

df_risk_factors_1["People without social security"]= [x.replace(',', '.') for x in df_risk_factors_1["People without social security"]]
df_risk_factors_1["People without social security"] = df_risk_factors_1["People without social security"].astype(float)

df_risk_factors_1.head(1)

,State Index,State,Total Population in the State,New Depression Cases (No per 100 thousand inhabitants),New Anorexia & Bulimia Cases (No per 100 thousand inhabitants),People with Water access at home,Percentage of population who has felt depressed,People migrating out of the State,People migrating inside the State,People without social security,Total Mortality by breast cancer,Total Mortality by Prostate Cancer,Total Population mortality by diabetes
0,1,Aguascalientes,1316032,107.32,1.58,1220535,20.2,233073,136354,12.9,71,69,801


### STEP 2 - INEGI -MEXICAN STATES - API QUERY DATA RETRIEVE AND CLEAN
#### The following tables were queried using INEGI's Website API.

##### We have previously selected codes from INEGI's Website, each state, municipality and feature have a proper code. We will use combinations of these codessuch as State-Feature or Municipality-Feature to perform the queries. Since the API gives only a single value per query, we're gonna have to automate the URL Query process to continue filling the Dataframe containing States, Municipalities and Features. In this case, every feature is a Risk Factor that increases the chances to trigger depression on any individual

#### API Query

In [6]:
!conda install furl

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - furl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.27 |                0         132 KB
    certifi-2019.11.28         |           py36_0         156 KB
    furl-2.1.0                 |             py_0          23 KB
    orderedmultidict-1.0.1     |           py36_0          21 KB
    ------------------------------------------------------------
                                           Total:         332 KB

The following NEW packages will be INSTALLED:

    furl:             2.1.0-py_0      
    orderedmultidict: 1.0.1-py36_0    

The following packages will be UPDATED:

    ca-certificates:  2019.10.16-0     --> 2019.11.27-0     
    certifi:          2019.9.11-py36_0 --> 2019.11.28-py36_0


ca-certificates-2019 | 132 KB    | ###

#### Create a list with the Feature, Municipality, State codes as stated by INEGI API so we can query the values and append them to a dataframe

In [84]:
#First we need to create a list with INEGI's Codes for each feature we want to append to the dataframe described before. 
#Since there were thousands of characteristics describing Mexico's population, we only chose 30 Features (Population's characteristics) related as risk
#factors for depression.Feature's name and its respected code were manually tabulated in excel and imported to IBM Watson Studio.

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='API Features-Municipality.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_api_features_M = pd.read_csv(body)

body = client_ddc00bb441b84f7792be0eedd381d2e5.get_object(Bucket='courseracapstoneprojectds-donotdelete-pr-vywtqbi6cifxib',Key='API Features-State.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_api_features_S = pd.read_csv(body)
df_api_features_S.head()


,Feature Name,Index Reference
0,People admitted for breach of trust to state p...,6200001464
1,People admitted sexual abuse to state prisons,6200001465
2,People declared being Discriminated,6200105772
3,Number of Workplace Conflicts legally registered,1007000012
4,Persons admitted for breach of obligations of ...,6200032446


In [83]:
#Then we will have to create a dataframe with INEGI's INDEX for each municipality and state, 
#We can retrieve this information from the first dataframe imported to this notebook called df_municipality

municipality_index= df_municipality['Municipality Index']
state_index= df_population_state['State Index']

#Now we can begin the URL API query

#### For Loop to fill the dataframes using the Feature and State Index

##### INEGI's API has a limit of 10 features and 1 State per query. So It will be needed to do a loop to query the info of all 32 States. Since the value to change in the URL is in the middle of the URL,"https://www.inegi.org.mx/value/value/value/VARIABLE1/value/value/VARIABLE2?=json" It was used a package known as "FURL" that allowed to separate in segments the URL query making it possible to make the For Loop. Moreover, some of INEGI's Index began by "0" like "01", which python transformed in "1" and insert it that way into the URL, giving a "Not Found 500" error, so it was needed to add a "zfill()" command to add those "0". Finally, while doing the For Loop, it was seen that some of the queries returned dictionaries with had a "None Type", so an IF ELIF Statement were used to set a ("0") when that happened

In [467]:
import requests
import json
from furl import furl
Observaciones=[]
features_index_S='6200001464,6200001465,6200105772,1007000012,6200032446,6200001478,6200032468,6200001504,6200240338,6300000264,6300000085'
df_risk_factors_2=df_api_features_S
f = furl('https://www.inegi.org.mx')
for state in state_index:
    if state < 10:
        state = str(state).zfill(2)
    Observaciones=[]
    Indicadores=[]
    f.path.segments = ['app','api','indicadores','desarrolladores','jsonxml','INDICATOR',features_index_S, 'es',state, 'true','BISE','2.0','1af978ce-b23a-f433-5f4e-0286c9a08144']
    f.args['type'] = 'json'
    f.url
    response= requests.get(f.url)
    if response.status_code==200:
        content= json.loads(response.content)
        Series=content['Series'][0]['OBSERVATIONS']
        Series1=content['Series'][1]['OBSERVATIONS']
        Series2=content['Series'][2]['OBSERVATIONS']
        Series3=content['Series'][3]['OBSERVATIONS']
        Series4=content['Series'][4]['OBSERVATIONS']
        Series5=content['Series'][5]['OBSERVATIONS']
        Series6=content['Series'][6]['OBSERVATIONS']
        Series7=content['Series'][7]['OBSERVATIONS']
        Series8=content['Series'][8]['OBSERVATIONS']
        Series9=content['Series'][9]['OBSERVATIONS']
        Series10=content['Series'][10]['OBSERVATIONS']
        for obs in Series:  
            Indicadores.append(content['Series'][0]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series1:  
            Indicadores.append(content['Series'][1]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series2:  
            Indicadores.append(content['Series'][2]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series3:  
            Indicadores.append(content['Series'][3]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series4:  
            Indicadores.append(content['Series'][4]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series5:  
            Indicadores.append(content['Series'][5]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series6:  
            Indicadores.append(content['Series'][6]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series7:  
            Indicadores.append(content['Series'][7]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series8:  
            Indicadores.append(content['Series'][8]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series9:  
            Indicadores.append(content['Series'][9]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series10:  
            Indicadores.append(content['Series'][10]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0);   
        df_Indicadores=pd.DataFrame(Indicadores)
        df_Indicadores=df_Indicadores.rename(columns= {0: "Index Reference"})
        df_Observaciones=pd.DataFrame(Observaciones)
        df_Observaciones=df_Observaciones.rename(columns= {0: str(state)})
        df_runner=pd.concat([df_Indicadores,df_Observaciones],axis=1)
        df_risk_factors_2=pd.merge(df_risk_factors_2.astype(str),df_runner.astype(str), on=['Index Reference'])
df_risk_factors_2



,Feature Name,Index Reference,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,People admitted for breach of trust to state p...,6200001464,19.0,51.0,7.0,5.0,33.0,11.0,31.0,15.0,0.0,5.0,85.0,42.0,30.0,18.0,25.0,6.0,2.0,52.0,13.0,6.0,13.0,24.0,5.0,23.0,64.0,227.0,1.0,14.0,19.0,35.0,15.0,26.0
1,People admitted sexual abuse to state prisons,6200001465,5.0,180.0,7.0,19.0,15.0,27.0,67.0,630.0,199.0,23.0,72.0,73.0,66.0,0.0,150.0,48.0,89.0,0.0,0.0,89.0,0.0,7.0,82.0,99.0,36.0,0.0,5.0,0.0,10.0,29.0,20.0,10.0
2,People declared being Discriminated,6200105772,9.0,203.0,2.0,8.0,14.0,88.0,72.0,221.0,145.0,8.0,22.0,64.0,247.0,0.0,381.0,41.0,6.0,7.0,11.0,105.0,153.0,102.0,250.0,65.0,13.0,0.0,125.0,128.0,99.0,218.0,0.0,0.0
3,Number of Workplace Conflicts legally registered,1007000012,3050.0,8704.0,1648.0,1465.0,10707.0,939.0,2654.0,11008.0,33842.0,2292.0,16461.0,5211.0,1878.0,22415.0,13959.0,5304.0,4022.0,1512.0,18250.0,1914.0,4629.0,8445.0,4514.0,4194.0,5195.0,10711.0,6176.0,5169.0,645.0,5582.0,4329.0,1076.0
4,Persons admitted for breach of obligations of ...,6200032446,34.0,460.0,103.0,77.0,17.0,87.0,781.0,56.0,36.0,5.0,94.0,650.0,409.0,0.0,88.0,25.0,24.0,0.0,21.0,5.0,0.0,197.0,121.0,72.0,167.0,1277.0,530.0,146.0,128.0,330.0,225.0,27.0
5,People admitted for other crimes against the f...,6200001478,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,164.0,13.0,0.0,0.0,3.0,4.0,3.0,312.0,1.0,0.0,0.0,0.0,55.0,2.0,4.0
6,People admitted gender-based violence in all i...,6200032468,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0
7,Persons admitted for family violence to state ...,6200001504,21.0,258.0,79.0,16.0,102.0,98.0,62.0,911.0,681.0,161.0,395.0,410.0,179.0,248.0,15.0,48.0,65.0,48.0,121.0,184.0,161.0,113.0,120.0,86.0,236.0,814.0,165.0,27.0,0.0,394.0,17.0,17.0
8,Number of Deaths by suicide,6200240338,140.0,194.0,57.0,69.0,216.0,51.0,147.0,365.0,227.0,122.0,504.0,63.0,109.0,630.0,584.0,245.0,88.0,66.0,317.0,140.0,319.0,138.0,125.0,204.0,135.0,267.0,142.0,164.0,55.0,253.0,235.0,99.0


#### Now we have to merge the new table with the DataFrame from STEP 1

##### For this purpose we will have to transpose the DataFrame and join them on State Index

In [543]:
df_risk_factors_2_T = df_risk_factors_2.copy()


In [544]:
#drop the column Index Reference and transpose the entire DataFrame, 
df_risk_factors_2_T.drop('Index Reference',axis=1,inplace=True)
df_risk_factors_2_T= df_risk_factors_2_T.transpose()
#then set the first row as column headers and drop the row
df_risk_factors_2_T.columns = df_risk_factors_2_T.iloc[0]
df_risk_factors_2_T.drop(df_risk_factors_2_T.index[0],inplace=True)
df_risk_factors_2_T.head()


Feature Name,People admitted for breach of trust to state prisons,People admitted sexual abuse to state prisons,People declared being Discriminated,Number of Workplace Conflicts legally registered,Persons admitted for breach of obligations of family assistance to state prisons,People admitted for other crimes against the family to state prisons,People admitted gender-based violence in all its different modalities to domestic violence to state prisons,Persons admitted for family violence to state prisons,Number of Deaths by suicide
01,19.0,5.0,9.0,3050.0,34.0,0.0,0.0,21.0,140.0
02,51.0,180.0,203.0,8704.0,460.0,0.0,0.0,258.0,194.0
03,7.0,7.0,2.0,1648.0,103.0,0.0,0.0,79.0,57.0
04,5.0,19.0,8.0,1465.0,77.0,0.0,0.0,16.0,69.0
05,33.0,15.0,14.0,10707.0,17.0,0.0,0.0,102.0,216.0


#### In order to merge this dataframe with the one in STEP 1, we are going to have to add another column as an index and change "Feature Name" to the real value of this column which is "State Index", then we will merge the columns on "State Index" of both dataframes.

In [545]:
#Add another column and alocate it as index column

df_risk_factors_2_T = df_risk_factors_2_T.reset_index()

#Rename the feature name column as "State Index"
df_risk_factors_2_T = df_risk_factors_2_T.rename(columns={"index":"State Index"})
#Change columns to float type, unless the "State Index"
df_risk_factors_2_T=df_risk_factors_2_T.astype(float)
df_risk_factors_2_T['State Index'] = df_risk_factors_2_T['State Index'].astype("int64")
#Drop the newly created column to set as index
df_risk_factors_2_T.head()

Feature Name,State Index,People admitted for breach of trust to state prisons,People admitted sexual abuse to state prisons,People declared being Discriminated,Number of Workplace Conflicts legally registered,Persons admitted for breach of obligations of family assistance to state prisons,People admitted for other crimes against the family to state prisons,People admitted gender-based violence in all its different modalities to domestic violence to state prisons,Persons admitted for family violence to state prisons,Number of Deaths by suicide
0,1,19.0,5.0,9.0,3050.0,34.0,0.0,0.0,21.0,140.0
1,2,51.0,180.0,203.0,8704.0,460.0,0.0,0.0,258.0,194.0
2,3,7.0,7.0,2.0,1648.0,103.0,0.0,0.0,79.0,57.0
3,4,5.0,19.0,8.0,1465.0,77.0,0.0,0.0,16.0,69.0
4,5,33.0,15.0,14.0,10707.0,17.0,0.0,0.0,102.0,216.0


#### Now we are capable of merging both DataFrames

In [546]:
df_risk_factors_3 = pd.merge(df_risk_factors_1,df_risk_factors_2_T, on=['State Index'])
df_risk_factors_3.head()


,State Index,State,Total Population in the State,New Depression Cases (No per 100 thousand inhabitants),New Anorexia & Bulimia Cases (No per 100 thousand inhabitants),People with Water access at home,Percentage of population who has felt depressed,People migrating out of the State,People migrating inside the State,People without social security,Total Mortality by breast cancer,Total Mortality by Prostate Cancer,Total Population mortality by diabetes,People admitted for breach of trust to state prisons,People admitted sexual abuse to state prisons,People declared being Discriminated,Number of Workplace Conflicts legally registered,Persons admitted for breach of obligations of family assistance to state prisons,People admitted for other crimes against the family to state prisons,People admitted gender-based violence in all its different modalities to domestic violence to state prisons,Persons admitted for family violence to state prisons,Number of Deaths by suicide
0,1,Aguascalientes,1316032,107.32,1.58,1220535,20.2,233073,136354,12.9,71,69,801,19.0,5.0,9.0,3050.0,34.0,0.0,0.0,21.0,140.0
1,2,Baja California,3348898,144.81,5.11,3000950,21.2,1299773,185457,18.1,192,164,2427,51.0,180.0,203.0,8704.0,460.0,0.0,0.0,258.0,194.0
2,3,Baja California Sur,718384,112.67,2.20,557229,21.2,246685,33074,13.6,34,36,377,7.0,7.0,2.0,1648.0,103.0,0.0,0.0,79.0,57.0
3,4,Campeche,902250,139.15,1.03,534468,31.3,180252,109734,11.9,35,28,723,5.0,19.0,8.0,1465.0,77.0,0.0,0.0,16.0,69.0
4,5,Coahuila de Zaragoza,2961708,31.23,0.57,2633210,45.6,362707,434617,14.1,222,136,2744,33.0,15.0,14.0,10707.0,17.0,0.0,0.0,102.0,216.0


### STEP 3 - INEGI -MEXICAN MUNICIPALITIES - API QUERY DATA RETRIEVE AND CLEAN
#### The following tables were queried using INEGI's Website API.

In [138]:
#Then we will have to create a dataframe with INEGI's INDEX for each municipality and state, 
#We can retrieve this information from the first dataframe imported to this notebook called df_municipality
import requests
import json
municipality_index= df_municipality['Municipality Index']
headers = requests.utils.default_headers()
headers['User-Agent'] = 'J NERI'

#Now we can begin the URL API query

In [ ]:

from furl import furl
Observaciones=[]
features_index_M='1002000030,1002000039,6207019060,6207019037,6207019029,6207019069,6207019042'
df_risk_factors_4=df_api_features_M.copy()
f = furl('https://www.inegi.org.mx')
for municipality in municipality_index:
    if municipality < 10000:
        municipality = str(municipality).zfill(5)
    Observaciones=[]
    Indicadores=[]
    f.path.segments = ['app','api','indicadores','desarrolladores','jsonxml','INDICATOR',features_index_M, 'es',municipality, 'true','BISE','2.0','d806d72d-d8f7-7337-ecc6-6c4c32940619?type=json']
    f.args['type'] = 'json'
    f.url
    response= requests.get(f.url,headers)
    if response.status_code==200:
        content= json.loads(response.content)
        Series=content['Series'][0]['OBSERVATIONS']
        Series1=content['Series'][1]['OBSERVATIONS']
        Series2=content['Series'][2]['OBSERVATIONS']
        Series3=content['Series'][3]['OBSERVATIONS']
        Series4=content['Series'][4]['OBSERVATIONS']
        Series5=content['Series'][5]['OBSERVATIONS']
        Series6=content['Series'][6]['OBSERVATIONS']
        for obs in Series:  
            Indicadores.append(content['Series'][0]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series1:  
            Indicadores.append(content['Series'][1]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series2:  
            Indicadores.append(content['Series'][2]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series3:  
            Indicadores.append(content['Series'][3]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series4:  
            Indicadores.append(content['Series'][4]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series5:  
            Indicadores.append(content['Series'][5]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0)
        for obs in Series6:  
            Indicadores.append(content['Series'][6]['INDICADOR'])
            if str(obs['OBS_VALUE'])!='None':
                Observaciones.append(float(obs['OBS_VALUE']))
            elif str(obs['OBS_VALUE'])=='None':
                Observaciones.append(0);   
        df_Indicadores2=pd.DataFrame(Indicadores)
        df_Indicadores2=df_Indicadores2.rename(columns= {0: "Index Reference"})
        df_Observaciones2=pd.DataFrame(Observaciones)
        df_Observaciones2=df_Observaciones2.rename(columns= {0: str(municipality)})
        df_runner2=pd.concat([df_Indicadores2,df_Observaciones2],axis=1)
        df_risk_factors_4=pd.merge(df_risk_factors_4.astype(str),df_runner2.astype(str), on=['Index Reference'])
        df_risk_factors_4.drop(df_risk_factors_4.index[7],inplace=True)
    print(municipality)
df_risk_factors_4


In [131]:
df_risk_factors_4

,Feature Name,Index Reference
0,Number of Deaths by Natural Cause,1002000030
1,Number of Divorces,1002000039
2,Percentage of Retired Employees,6207019060
3,Percentage of homes lacking a proper roof,6207019037
4,Percentage of population between 14-25 with ac...,6207019029
5,Percentage of population between 14-25 that ha...,6207019069
6,Percentage of homes with internet access,6207019042


In [137]:
url='https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000002/es/00000/false/BISE/2.0/d806d72d-d8f7-7337-ecc6-6c4c32940619?type=json'
response= requests.get(url)
response.status_code


200

In [110]:
Observaciones=[]
Indicadores=[]
content= json.loads(response.content)
Series=content['Series'][0]['OBSERVATIONS']
Series1=content['Series'][1]['OBSERVATIONS']
Series2=content['Series'][2]['OBSERVATIONS']
Series3=content['Series'][3]['OBSERVATIONS']
Series4=content['Series'][4]['OBSERVATIONS']
Series5=content['Series'][5]['OBSERVATIONS']
Series6=content['Series'][6]['OBSERVATIONS']
for obs in Series:
    Indicadores.append(content['Series'][0]['INDICADOR'])
    if str(obs['OBS_VALUE'])!='None':
        Observaciones.append(float(obs['OBS_VALUE']))
    elif str(obs['OBS_VALUE'])=='None':
        Observaciones.append(0)
for obs in Series1:  
    Indicadores.append(content['Series'][1]['INDICADOR'])
    if str(obs['OBS_VALUE'])!='None':
        Observaciones.append(float(obs['OBS_VALUE']))
    elif str(obs['OBS_VALUE'])=='None':
        Observaciones.append(0)
for obs in Series2: 
    Indicadores.append(content['Series'][2]['INDICADOR'])
    if str(obs['OBS_VALUE'])!='None':
        Observaciones.append(float(obs['OBS_VALUE']))
    elif str(obs['OBS_VALUE'])=='None':
        Observaciones.append(0)
for obs in Series3:  
    Indicadores.append(content['Series'][3]['INDICADOR'])
    if str(obs['OBS_VALUE'])!='None':
        Observaciones.append(float(obs['OBS_VALUE']))
    elif str(obs['OBS_VALUE'])=='None':
        Observaciones.append(0)
for obs in Series4:  
    Indicadores.append(content['Series'][4]['INDICADOR'])
    if str(obs['OBS_VALUE'])!='None':
        Observaciones.append(float(obs['OBS_VALUE']))
    elif str(obs['OBS_VALUE'])=='None':
        Observaciones.append(0)
for obs in Series5:  
    Indicadores.append(content['Series'][5]['INDICADOR'])
    if str(obs['OBS_VALUE'])!='None':
        Observaciones.append(float(obs['OBS_VALUE']))
    elif str(obs['OBS_VALUE'])=='None':
        Observaciones.append(0)
for obs in Series6:  
    Indicadores.append(content['Series'][6]['INDICADOR'])
    if str(obs['OBS_VALUE'])!='None':
        Observaciones.append(float(obs['OBS_VALUE']))
    elif str(obs['OBS_VALUE'])=='None':
        Observaciones.append(0);   
df_Indicadores2=pd.DataFrame(Indicadores)
df_Indicadores2=df_Indicadores2.rename(columns= {0: "Index Reference"})
df_Observaciones2=pd.DataFrame(Observaciones)
df_Observaciones2=df_Observaciones2.rename(columns= {0: str(municipality)})
df_runner2=pd.concat([df_Indicadores2,df_Observaciones2],axis=1)
test=df_api_features_M.copy()
test=pd.merge(test.astype(str),df_runner2.astype(str), on=['Index Reference'])
test.drop(test.index[7],inplace=True)
test

,Feature Name,Index Reference,01001
0,Number of Deaths by Natural Cause,1002000030,4250.0
1,Number of Divorces,1002000039,2767.0
2,Percentage of Retired Employees,6207019060,9.3007555
3,Percentage of homes lacking a proper roof,6207019037,0.0742536
4,Percentage of population between 14-25 with ac...,6207019029,46.159389
5,Percentage of population between 14-25 that ha...,6207019069,0.5219308
6,Percentage of homes with internet access,6207019042,42.075713


In [116]:
df_api_features_M

,Feature Name,Index Reference
0,Number of Deaths by Natural Cause,1002000030
1,Number of Divorces,1002000039
2,Percentage of Retired Employees,6207019060
3,Percentage of homes lacking a proper roof,6207019037
4,Percentage of population between 14-25 with ac...,6207019029
5,Percentage of population between 14-25 that ha...,6207019069
6,Percentage of homes with internet access,6207019042
